# Test of DiffVis

## Settings

In [1]:
import functools
import html
import pathlib
from pprint import pprint
import re
import sys
import unicodedata

In [2]:
import MeCab
from IPython.display import HTML

In [3]:
sys.path.append('../')
from DiffVis.diffvis import DiffVis

## Define

In [4]:
class Tokenizer(object):
    """Tokenize Japanese sentence to words.

    Args:
        mecab_dict_path (str or pathlib.Path): Path to MeCab dictionary.
        parts_of_speech (list[str]): List that contains part of speeches to be extracted.
            If None (default), these are used:
                [
                    '名詞', '動詞', '形容詞', '副詞', '助詞',
                    '接続詞', '助動詞', '連体詞', '感動詞',
                ]
        normalize (bool): Flag to determine whether to transform to base form or not.
            Defaults to False.

    Attributes:
        tagger (MeCab.Tagger): Tagger.
        parts_of_speech (list[str]): List that contains part of speeches to be extracted.
        normalize (bool): Flag to determine whether to transform to base form or not.
    """
    def __init__(self, mecab_dict_path='', parts_of_speech=None, normalize=False,):
        added = ''
        if mecab_dict_path:
            added = ' -d' + str(mecab_dict_path)
        self.tagger = MeCab.Tagger('-Ochasen' + added)
        self.tagger.parse('')
        if parts_of_speech:
            self.parts_of_speech = parts_of_speech
        else:
            self.parts_of_speech = [
                '名詞', '動詞', '形容詞', '副詞', '助詞',
                '接続詞', '助動詞', '連体詞', '感動詞',
                ]
        self.normalize = normalize

    def tokenize(self, sentence):
        """Tokenize Japanese sentence to words.

        Args:
            sentence (str): Sentence to tokenize.

        Returns:
            words (list[str]): Extracted words.
        """
        res = self.tagger.parseToNode(sentence)
        words = []
        if self.normalize:
            while res:
                content = res.feature.split(',')
                part_of_speech = content[0]
                if part_of_speech in self.parts_of_speech:
                    word = content[6]
                    if word == '*':
                        word = res.surface
                    words.append(word)
                res = res.next
        else:
            while res:
                word = res.surface
                part_of_speech = res.feature.split(',')[0]
                if part_of_speech in self.parts_of_speech:
                    words.append(word)
                res = res.next
        return words

## Toy Data

### Simple string

In [5]:
source = 'すももも桃も桃のうち1234'
target = 'すもももももももものうち1244'
m, n = len(source), len(target)
print(m, n)

14 16


In [6]:
dv = DiffVis(source, target)
dv.build()

In [7]:
print(dv.distance(normalize=True))
print(dv.distance(normalize=False))

0.3125
5


In [32]:
dv.cost_table

((0, 1, 2, 3, 4, 5, 6, 7),
 (1, 0, 1, 2, 3, 4, 5, 6),
 (2, 1, 0, 1, 2, 3, 4, 5),
 (3, 2, 1, 1, 2, 3, 4, 5),
 (4, 3, 2, 2, 1, 2, 3, 4),
 (5, 4, 3, 3, 2, 2, 3, 4),
 (6, 5, 4, 4, 3, 3, 2, 3),
 (7, 6, 5, 5, 4, 4, 3, 2))

In [9]:
print(dv.format_edit_history())

Edit History
	match
	match
	match
	match
	replace
	match
	replace
	insert
	insert
	match
	match
	match
	match
	match
	replace
	match


In [10]:
print(dv.generate_comparison(mode='console', padding=True))

すももも桃も桃　　のうち 1 2 3 4
すもももももももものうち 1 2 4 4


In [11]:
print(dv.generate_comparison(mode='console', padding=False))

すももも桃も桃のうち1234
すもももももももものうち1244


In [12]:
HTML(dv.generate_comparison(mode='html', padding=True))

In [13]:
HTML(dv.generate_comparison(mode='html', padding=False))

In [14]:
HTML(dv.generate_comparison(mode='htmltab'))

す,も,も,も,桃,も,桃,,,の,う,ち,1,2,3,4
す,も,も,も,も,も,も,も,も,の,う,ち,1,2,4,4


In [15]:
dv.template(use_str=True)

'すももも<blank>も<blank>のうち12<blank>4'

In [16]:
dv.template()

['す',
 'も',
 'も',
 'も',
 '<blank>',
 'も',
 '<blank>',
 'の',
 'う',
 'ち',
 '1',
 '2',
 '<blank>',
 '4']

### Tokenize

In [17]:
tokenizer = Tokenizer()

In [18]:
source = 'すももも桃も桃のうち'
target = 'すもももももももものうち'
m, n = len(source), len(target)
print(m, n)

10 12


In [19]:
source = tokenizer.tokenize(source)
target = tokenizer.tokenize(target)
m, n = len(source), len(target)
print(m, n)

7 7


In [20]:
dv = DiffVis(source, target)
dv.build()

In [21]:
print(dv.distance(normalize=True))
print(dv.distance(normalize=False))

0.2857142857142857
2


In [22]:
dv.cost_table

((0, 1, 2, 3, 4, 5, 6, 7),
 (1, 0, 1, 2, 3, 4, 5, 6),
 (2, 1, 0, 1, 2, 3, 4, 5),
 (3, 2, 1, 1, 2, 3, 4, 5),
 (4, 3, 2, 2, 1, 2, 3, 4),
 (5, 4, 3, 3, 2, 2, 3, 4),
 (6, 5, 4, 4, 3, 3, 2, 3),
 (7, 6, 5, 5, 4, 4, 3, 2))

In [23]:
print(dv.format_edit_history())

Edit History
	match
	match
	replace
	match
	replace
	match
	match


In [24]:
print(dv.generate_comparison(mode='console', padding=True))

すももも桃　も桃　のうち
すもももももももものうち


In [25]:
print(dv.generate_comparison(mode='console', padding=False))

すももも桃も桃のうち
すもももももももものうち


In [26]:
HTML(dv.generate_comparison(mode='html', padding=True))

In [27]:
HTML(dv.generate_comparison(mode='html', padding=False))

In [28]:
HTML(dv.generate_comparison(mode='htmltab'))

すもも,も,桃,も,桃,の,うち
すもも,も,もも,も,もも,の,うち


In [29]:
dv.template(use_str=True)

'すももも<blank>も<blank>のうち'

In [30]:
dv.template()

['すもも', 'も', '<blank>', 'も', '<blank>', 'の', 'うち']

## Test of Levenshtein Distance

In [31]:
!python edit_distance.py source target -a

Levenshtein Distance: 5
Normalized Levenshtein Distance: 0.833

Cost Table
---------------------------------
|   |   | t | a | r | g | e | t |
---------------------------------
|   | 0 | 1 | 2 | 3 | 4 | 5 | 6 |
---------------------------------
| s | 1 | 1 | 2 | 3 | 4 | 5 | 6 |
---------------------------------
| o | 2 | 2 | 2 | 3 | 4 | 5 | 6 |
---------------------------------
| u | 3 | 3 | 3 | 3 | 4 | 5 | 6 |
---------------------------------
| r | 4 | 4 | 4 | 3 | 4 | 5 | 6 |
---------------------------------
| c | 5 | 5 | 5 | 4 | 4 | 5 | 6 |
---------------------------------
| e | 6 | 6 | 6 | 5 | 5 | 4 | 5 |
---------------------------------

Edit History
	replace
	replace
	delete
	match
	replace
	match
	insert
